<a href="https://colab.research.google.com/github/JUANPABLO-0826/TravelInsight/blob/main/travel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### TRAVEL INSIGHT

In [4]:
# con esto Instalas la libreria
!pip install pandas
!pip install flask flask-ngrok
!pip install pyngrok

In [5]:
from pyngrok import conf
# Configurar ngrok authtoken
conf.get_default().auth_token = "30KVIM2dHFxcJjsTBq9va5jyszC_65wEZww5VZKwqiEYTEmrg"



In [6]:
from flask import Flask, jsonify
from pyngrok import ngrok
import pandas as pd

# Cargar datos desde GitHub
url = "https://raw.githubusercontent.com/MunchiP/TravleInsight/refs/heads/JuanPablo/Travel/rnt_limpio.csv"
df = pd.read_csv(url)

# Crear la app
app = Flask(__name__)

@app.route("/")
def index():
    return "✅ API Travel Insight funcionando"

# /api/destinos → primeros 10 registros
@app.route("/api/destinos")
def destinos():
    data = df.head(10).to_dict(orient="records")
    return jsonify(data)

# /api/municipios → lista de municipios únicos
@app.route("/api/municipios")
def municipios():
    municipios = df["MUNICIPIO"].dropna().unique().tolist()
    return jsonify(municipios)

# /api/categorias → lista de categorías únicas
@app.route("/api/categorias")
def categorias():
    cats = df["CATEGORIA"].dropna().unique().tolist()
    return jsonify(cats)

@app.route("/api/destinos/<municipio>")
def destinos_por_municipio(municipio):
    try:
        # Eliminamos NaN en MUNICIPIO para evitar errores
        df_validos = df.dropna(subset=["MUNICIPIO"])

        # Filtramos ignorando mayúsculas y tildes
        filtro = df_validos[df_validos["MUNICIPIO"].str.lower() == municipio.lower()]

        # Devuelve los resultados
        return jsonify(filtro.to_dict(orient="records"))
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# /api/top-municipios → top 5 municipios con más registros
@app.route("/api/top-municipios")
def top_municipios():
    try:
        top = df["MUNICIPIO"].value_counts().head(5).to_dict()
        return jsonify(top)
    except Exception as e:
        return jsonify({"error": str(e)}), 500


# Filtrar por categoría
@app.route("/api/categoria/<nombre>")
def por_categoria(nombre):
    try:
        filtro = df[df["CATEGORIA"].str.lower() == nombre.lower()]
        return jsonify(filtro.to_dict(orient="records"))
    except Exception as e:
        return jsonify({"error": str(e)}), 500


# Crear túnel público
public_url = ngrok.connect(5000)
print("🔗 Tu API está disponible en:", public_url)

# Ejecutar la app
app.run(port=5000)

🔗 Tu API está disponible en: NgrokTunnel: "https://327bb871c0af.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [7]:
print(df["MUNICIPIO"].value_counts().head(5))


MUNICIPIO
VILLA DE LEYVA    896
RICAURTE          743
GIRARDOT          643
VILLETA           478
PAIPA             452
Name: count, dtype: int64


In [8]:
print(df["CATEGORIA"].value_counts().head(5))

CATEGORIA
VIVIENDAS TURÍSTICAS                         6880
ESTABLECIMIENTOS DE ALOJAMIENTO TURÍSTICO    2401
AGENCIAS DE VIAJES                           1400
GUIAS DE TURISMO                              529
ESTABLECIMIENTOS DE GASTRONOMÍA               118
Name: count, dtype: int64
